In [71]:
import numpy as np

In [121]:
SUN_RANK = [7, 8, 9, 11, 12, 13, 10, 1]

In [113]:
np.random.seed(42)

# SUIT ORDER: HEARTS, SPADES, DIAMONDS, CLUBS

player_cards = np.array([rank + suit * 13 for suit in (0, 1, 2, 3) for rank in (1, 7, 8, 9, 10, 11, 12, 13)], dtype='uint8')
np.random.shuffle(player_cards)
player_cards = player_cards.reshape(4, 8)

player_cards

array([[50, 26, 40, 33, 14, 20, 51, 46],
       [23,  1, 10, 27, 11, 24, 22, 39],
       [ 7,  8, 47,  9, 37, 48, 38, 34],
       [52, 36, 13, 21, 25, 49, 35, 12]], dtype=uint8)

In [125]:
def get_suit(card):
    return (card - 1) // 13

def get_rank(card):
    return (card - 1) % 13 + 1

def has_suit(cards, suit):
    for card in cards:
        if get_suit(card) == suit:
            return True
    return False

def get_trick_winner(trick):
    trick_suit = get_suit(trick[0])
    
    winner = 0
    max_rank_idx = SUN_RANK.index(get_rank(trick[0]))
    for i, card in enumerate(trick):
        if get_suit(card) == trick_suit and SUN_RANK.index(get_rank(trick[i])) > max_rank_idx:
            max_rank_idx = SUN_RANK.index(get_rank(trick[i]))
            winner = i

    return winner

In [159]:
class State:
    def __init__(self, played_cards, current_cards, current_player, num_of_played_cards, trick_starter, parent):
        self.state_score = 0
        self.state_visits = 0
        self.played_cards = played_cards # 1D array of cards played in order
        self.current_cards = current_cards # 2D array of the remaining cards in each player's hand
        self.current_player = current_player
        self.num_of_played_cards = num_of_played_cards
        self.trick_starter = trick_starter
        self.child_states = {
            # action: new_state
        }
        self.parent = parent

    def expand(self):
        if self.num_of_played_cards % 4 == 0:
            # first card in the trick
            for i, card in enumerate(self.current_cards[self.current_player]):
                if card == 0:
                    continue
                    
                updated_current_cards = self.current_cards.copy()
                updated_current_cards[self.current_player][i] = 0

                updated_played_cards = self.played_cards.copy()
                updated_played_cards[self.num_of_played_cards] = card

                self.child_states[card] = State(updated_played_cards, updated_current_cards, self.current_player + 1 % 4,
                                                self.num_of_played_cards + 1, self.trick_starter, self)

        else:
            trick_first_card = self.played_cards[(self.num_of_played_cards // 4) * 4]
            trick_suit = get_suit(trick_first_card)

            player_has_suit = has_suit(self.current_cards[self.current_player], trick_suit)
            for i, card in enumerate(self.current_cards[self.current_player]):
                if player_has_suit and trick_suit == get_suit(card) or not player_has_suit:
                    if card == 0:
                        continue
                        
                    updated_current_cards = self.current_cards.copy()
                    updated_current_cards[self.current_player][i] = 0
    
                    updated_played_cards = self.played_cards.copy()
                    updated_played_cards[self.num_of_played_cards] = card

                    if self.num_of_played_cards % 4 == 3: # last card in the trick
                        updated_trick_starter = get_trick_winner(updated_played_cards[self.num_of_played_cards - 3: self.num_of_played_cards + 1]) + self.trick_starter % 4
                    
                        self.child_states[card] = State(updated_played_cards, updated_current_cards, updated_trick_starter,
                                                    self.num_of_played_cards + 1, updated_trick_starter, self)
                    else:
                        self.child_states[card] = State(updated_played_cards, updated_current_cards, self.current_player + 1 % 4,
                                                self.num_of_played_cards + 1, self.trick_starter, self)

In [209]:
initial_state = State(played_cards=np.zeros(32, dtype='uint8'), current_cards=player_cards.copy(), current_player=3, num_of_played_cards=3,
                     trick_starter=0, parent=None)

In [213]:
initial_state.expand()
initial_state.child_states

{52: <__main__.State at 0x1273f6240>,
 36: <__main__.State at 0x1273f7f80>,
 13: <__main__.State at 0x1273f4aa0>,
 21: <__main__.State at 0x1273f6150>,
 25: <__main__.State at 0x1273f69f0>,
 49: <__main__.State at 0x1273f4260>,
 35: <__main__.State at 0x1273f4a40>,
 12: <__main__.State at 0x1273f6900>}